In [50]:
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [52]:
img_size = (299, 299)
batch_size = 32

In [53]:
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(299, scale=(0.8, 1.0)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [54]:
data_dir = "Cotton_Disease"
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

train_dataset = ImageFolder(train_dir, transform=transform)
val_dataset = ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

num_classes = len(train_dataset.classes)
print("Number of classes:", num_classes)

Number of classes: 4


In [55]:
vgg = models.vgg19(pretrained=True)
inception = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
inception.aux_logits = False 
resnet = models.resnet50(pretrained=True)

In [56]:
for model in [vgg, inception, resnet]:
    for param in model.parameters():
        param.requires_grad = False

In [57]:
vgg_fc = vgg.classifier[0].in_features
vgg.classifier = nn.Sequential(nn.Linear(vgg_fc, 512), nn.ReLU(), nn.Linear(512, num_classes))

inception_fc = inception.fc.in_features
inception.fc = nn.Sequential(nn.Linear(inception_fc, 512), nn.ReLU(), nn.Linear(512, num_classes))

resnet_fc = resnet.fc.in_features
resnet.fc = nn.Sequential(nn.Linear(resnet_fc, 512), nn.ReLU(), nn.Linear(512, num_classes))

vgg, inception, resnet = vgg.to(device), inception.to(device), resnet.to(device)

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self, vgg, inception, resnet, num_classes):
        super(EnsembleModel, self).__init__()
        self.vgg = vgg
        self.inception = inception
        self.resnet = resnet
        self.fc = nn.Linear(3 * num_classes, num_classes)
        
    def forward(self, x):
        vgg_out = self.vgg(x)
        inception_out = self.inception(x)
        resnet_out = self.resnet(x)

        combined = torch.cat((vgg_out, inception_out, resnet_out), dim=1)
        return self.fc(combined)

In [ ]:
model = EnsembleModel(vgg, inception, resnet, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    best_acc = 0.0
    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_acc = 100 * correct / total
        print(f"Validation Accuracy: {val_acc:.2f}%\n")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model, "best_model.pth")
            print("✅ Best Model Saved!\n")

In [61]:
train_model(model, train_loader, val_loader, criterion, optimizer)

Epoch [1/20], Loss: 39.3204, Train Accuracy: 76.47%
Validation Accuracy: 86.17%

✅ Best Model Saved!

Epoch [2/20], Loss: 11.7619, Train Accuracy: 93.59%
Validation Accuracy: 93.68%

✅ Best Model Saved!

Epoch [3/20], Loss: 7.9508, Train Accuracy: 95.54%
Validation Accuracy: 87.75%

Epoch [4/20], Loss: 8.1321, Train Accuracy: 95.44%
Validation Accuracy: 93.68%

Epoch [5/20], Loss: 5.6873, Train Accuracy: 96.62%
Validation Accuracy: 95.65%

✅ Best Model Saved!

Epoch [6/20], Loss: 5.5982, Train Accuracy: 96.67%
Validation Accuracy: 94.07%

Epoch [7/20], Loss: 4.9886, Train Accuracy: 97.13%
Validation Accuracy: 97.63%

✅ Best Model Saved!

Epoch [8/20], Loss: 3.4788, Train Accuracy: 98.15%
Validation Accuracy: 97.63%

Epoch [9/20], Loss: 4.8692, Train Accuracy: 97.13%
Validation Accuracy: 90.91%

Epoch [10/20], Loss: 3.1719, Train Accuracy: 98.36%
Validation Accuracy: 94.07%

Epoch [11/20], Loss: 2.5960, Train Accuracy: 98.46%
Validation Accuracy: 95.65%

Epoch [12/20], Loss: 2.0740, Tra